In [2]:
# Import Libraries
import numpy as np
import pandas as pd

from collections import defaultdict
import threading
import time

# Database Integration Libraries
import sqlite3
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import datetime

In [2]:
# !pip install SQLAlchemy 

In [3]:
# # A pre-defined menu of food and beverages
# menu = {
#     1: {"name": "Pizza", "price": 10.99},
#     2: {"name": "Burger", "price": 8.99},
#     3: {"name": "Pasta", "price": 12.99},
#     4: {"name": "Salad", "price": 6.99},
#     5: {"name": "Soda", "price": 1.99},
#     6: {"name": "Coffee", "price": 2.99},
# }

In [4]:
# # A Dict to store orders by food item:
# order_queue = defaultdict(list)

In [5]:
# # Function to dispatch orders after 15 seconds
# def dispatch_orders(food_item):
#     print(f"\nDispatching order for {food_item}:")
#     for order in order_queue[food_item]:
#         print(f" - Deliver to Table {order['table']} (ordered at {time.ctime(order['timestamp'])})")
#     # Clear the orders for the food item after dispatch
#     order_queue[food_item].clear()

# # Function to queue order and set 15-second timer for dispatch
# def queue_order(food_item, table):
#     current_time = time.time()
#     order = {"table": table, "timestamp": current_time}
#     # Add the order to the queue for the specified food item
#     order_queue[food_item].append(order)
#     # If this is the first order for this item, start a 15-second timer to dispatch
#     if len(order_queue[food_item]) == 1:
#         print(f"\nNew order received for {food_item}. Starting a 15-second countdown.")
#         threading.Timer(0.25 * 60, dispatch_orders, [food_item]).start()  # 15-second countdown
#     else:
#         print(f"Similar order for {food_item} added to existing batch.")

# # Function to display the menu
# def display_menu():
#     print("\n--- Menu ---")
#     for item_id, details in menu.items():
#         print(f"{item_id}. {details['name']} - ${details['price']:.2f}")

# # Function to take an order from the customer
# def take_order():
#     display_menu()
#     try:
#         item_number = int(input("\nEnter the number of the food item you want to order: "))
#         if item_number not in menu:
#             print("Invalid selection. Please choose a valid item from the menu.")
#             return
#         table = input("Enter Table number: ")
#         food_item = menu[item_number]["name"]
#         queue_order(food_item, table)
#         print(f"Order for {food_item} at table {table} has been placed.")
#     except ValueError:
#         print("Please enter a valid number.")

# # User-driven interaction for managing orders
# def simulation():
#     while True:
#         print("\nChoose an action:")
#         print("1. Place an Order")
#         print("2. View Active Orders")
#         print("3. Exit Simulation")

#         choice = input("\nEnter choice (1-3): ")

#         if choice == '1':
#             take_order()
#         elif choice == '2':
#             view_active_orders()
#         elif choice == '3':
#             print("\nExiting simulation.")
#             break
#         else:
#             print("\nInvalid choice. Please try again.")

# # Function to view all active orders
# def view_active_orders():
#     if not order_queue:
#         print("\nNo active orders.")
#     else:
#         for food_item, orders in order_queue.items():
#             print(f"\n{food_item} Orders:")
#             for order in orders:
#                 print(f" - Table {order['table']}, ordered at {time.ctime(order['timestamp'])}")


In [6]:
# # Run the improved simulation
# simulation()

In [3]:
# A defaultdict to store current orders by food item before dispatch
order_queue = defaultdict(list)

# Menu for food items and prices
menu = {
    'Burger': 5.00,
    'Pizza': 8.00,
    'Pasta': 6.50,
    'Salad': 4.50,
    'Soda': 1.50
}

# Database connection for storing orders
conn = sqlite3.connect('orders.db')
cursor = conn.cursor()

# Create table to store orders if it doesn't already exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS orders (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    food_item TEXT,
    table_number TEXT,
    timestamp REAL,
    status TEXT
)
''')
conn.commit()

# Function to dispatch orders after the timer
def dispatch_orders(food_item):
    print(f"\nDispatching orders for {food_item}:")
    for order in order_queue[food_item]:
        print(f" - Deliver to Table {order['table']} (ordered at {time.ctime(order['timestamp'])})")
        cursor.execute("UPDATE orders SET status = 'dispatched' WHERE timestamp = ?", (order['timestamp'],))
    # Clear the order queue for that food item
    order_queue[food_item].clear()
    conn.commit()

# Function to queue new orders and dispatch after a delay
def queue_order(food_item, table):
    current_time = time.time()
    order = {"table": table, "timestamp": current_time}
    # Add the order to the queue for that food item
    order_queue[food_item].append(order)
    # Save to the database with pending status
    cursor.execute("INSERT INTO orders (food_item, table_number, timestamp, status) VALUES (?, ?, ?, 'pending')", 
                   (food_item, table, current_time))
    conn.commit()
    # Start a timer if it's the first order for that food item
    if len(order_queue[food_item]) == 1:
        print(f"New order received for {food_item}. Dispatching in 5 minutes.")
        threading.Timer(5 * 60, dispatch_orders, [food_item]).start()  # Change to 5 minutes (300 seconds)
    else:
        print(f"Another order for {food_item} added to the batch.")

# Function to take an order from the customer
def display_menu():
    print("\n--- Menu ---")
    for item, price in menu.items():
        print(f"{item}: ${price}")

def take_order():
    display_menu()
    food_item = input("Select a food item from the menu: ")
    if food_item not in menu:
        print("Invalid item. Please choose from the menu.")
        return
    table = input("Enter Table number: ")
    queue_order(food_item, table)
    print(f"Order for {food_item} at table {table} has been placed.")

# Admin panel for managing orders and viewing history
def view_order_history():
    print("\n--- Order History ---")
    cursor.execute("SELECT * FROM orders ORDER BY timestamp")
    orders = cursor.fetchall()
    if orders:
        for order in orders:
            status = "Pending" if order[4] == 'pending' else "Dispatched"
            print(f"Order ID: {order[0]}, Food: {order[1]}, Table: {order[2]}, Time: {time.ctime(order[3])}, Status: {status}")
    else:
        print("No orders found.")

def generate_summary_report():
    print("\n--- Order Summary Report ---")
    cursor.execute("SELECT food_item, COUNT(*), SUM(CASE WHEN status='dispatched' THEN 1 ELSE 0 END) as dispatched_count FROM orders GROUP BY food_item")
    report = cursor.fetchall()
    if report:
        for row in report:
            print(f"Food Item: {row[0]}, Total Orders: {row[1]}, Dispatched: {row[2]}")
    else:
        print("No orders available to summarize.")

def admin_menu():
    while True:
        print("\n--- Admin Panel ---")
        print("1. View Order History")
        print("2. Generate Summary Report")
        print("3. Exit Admin Panel")
        choice = input("Select an option: ")
        
        if choice == '1':
            view_order_history()
        elif choice == '2':
            generate_summary_report()
        elif choice == '3':
            print("Exiting Admin Panel.")
            break
        else:
            print("Invalid option. Try again.")

# Main loop to simulate the restaurant app
def main():
    while True:
        print("\n--- DishDispatch App ---")
        print("1. Take an Order")
        print("2. Admin Panel")
        print("3. Exit App")
        choice = input("Select an option: ")
        
        if choice == '1':
            take_order()
        elif choice == '2':
            admin_menu()
        elif choice == '3':
            print("Exiting DishDispatch App.")
            break
        else:
            print("Invalid choice. Try again.")

if __name__ == "__main__":
    main()

# Close the database connection on exit
conn.close()



--- DishDispatch App ---
1. Take an Order
2. Admin Panel
3. Exit App

--- Menu ---
Burger: $5.0
Pizza: $8.0
Pasta: $6.5
Salad: $4.5
Soda: $1.5
Invalid item. Please choose from the menu.

--- DishDispatch App ---
1. Take an Order
2. Admin Panel
3. Exit App
Invalid choice. Try again.

--- DishDispatch App ---
1. Take an Order
2. Admin Panel
3. Exit App
Invalid choice. Try again.

--- DishDispatch App ---
1. Take an Order
2. Admin Panel
3. Exit App
Invalid choice. Try again.

--- DishDispatch App ---
1. Take an Order
2. Admin Panel
3. Exit App
Invalid choice. Try again.

--- DishDispatch App ---
1. Take an Order
2. Admin Panel
3. Exit App
Invalid choice. Try again.

--- DishDispatch App ---
1. Take an Order
2. Admin Panel
3. Exit App
Invalid choice. Try again.

--- DishDispatch App ---
1. Take an Order
2. Admin Panel
3. Exit App
Invalid choice. Try again.

--- DishDispatch App ---
1. Take an Order
2. Admin Panel
3. Exit App
Exiting DishDispatch App.
